In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import scipy as scipy

# Create graph



In [ ]:
G = nx.DiGraph()

[G.add_node(k) for k in ["A", "B", "C", "D", "E", "F", "G"]]
G.add_edges_from([('G','A'), ('A','G'),('B','A'),
                  ('C','A'),('A','C'),('A','D'),
                  ('E','A'),('F','A'),('B','D'),
                  ('D','F')])
pos = nx.spiral_layout(G)
nx.draw(G, pos, with_labels = True, node_color="red")

# Run pagerank

In [ ]:
pr1 = nx.pagerank(G)
print(pr1)
nx.draw(G, pos, nodelist=list(pr1.keys()), node_size=[round(v * 4000) for v in pr1.values()], 
        with_labels = True, node_color="red")   

# Adjusting dampening/teleports

In [ ]:
pr_09 = nx.pagerank(G, alpha=0.9)
pr_08 = nx.pagerank(G, alpha=0.8)
res = pd.DataFrame({"alpha=0.9": pr_09, "alpha=0.8": pr_08})
res

# Personal PageRank

In [ ]:
pr_e = nx.pagerank(G, alpha=0.9, personalization={'A': 1})
print(pr_e)
nx.draw(G, pos, nodelist=list(pr_e.keys()), node_size=[round(v * 4000) for v in pr_e.values()], 
        with_labels = True, node_color="red")   

# MovieLens Recommender

## Load data

In [ ]:
import urllib.request
import shutil
import zipfile
import os
filename='ml-100k'
data_url='https://files.grouplens.org/datasets/movielens/ml-100k.zip'
with urllib.request.urlopen(data_url) as response, open('./'+filename, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
print('Download completed')
with zipfile.ZipFile('./'+filename, 'r') as zip_ref:
    zip_ref.extractall('./sample_data/')
dirs = [x[0] for x in os.walk("./sample_data")]
ml = filter(lambda dirName: dirName if ('ml' in dirName) else '', list(dirs))
dt_dir_name= list(ml)[0]
rdata = pd.read_csv(dt_dir_name +'/'+ 'u.data', delimiter='\t', names=['userId', 'movieId', 'rating', 'timestamp'])
rdata['userId'] = 'u' + rdata['userId'].astype(str)
rdata['movieId'] = 'i' + rdata['movieId'].astype(str)

usrdata = pd.read_csv(dt_dir_name +'/'+'u.user', delimiter='|', names=['user id', 'age' ,'gender' ,'occupation' , 'zip code'])
item_data = pd.read_csv(dt_dir_name +'/'+ 'u.item', delimiter='|', encoding="ISO-8859-1", header=None)
item_data = item_data[[0,1]]
item_data.columns = ['movieId','movieTitle']
item_data['movieId'] = 'i' + item_data['movieId'].astype(str)
item_data = item_data.set_index('movieId')
rdata.head()

In [ ]:
rdata = pd.merge(rdata, item_data, how='left', on='movieId')
rdata.head()

## Create graph

In [ ]:
#Create a graph
G = nx.Graph()
#Add nodes
G.add_nodes_from(rdata.userId, bipartite=0)
G.add_nodes_from(rdata.movieId, bipartite=1)
#Add weights for edges
G.add_weighted_edges_from([(uId, mId,rating) for (uId, mId, rating)
              in rdata[['userId', 'movieId', 'rating']].to_numpy()])
print(nx.info(G))
print(nx.is_bipartite(G))

## Run pagerank

In [ ]:
movie_rank = nx.pagerank(G, alpha=0.85)

In [ ]:
def return_top_movies(movie_rank):
  movie_rank = dict(sorted(movie_rank.items(), key=lambda item: item[1], reverse=True))
  top_10_movies = []
  for key, value in movie_rank.items():
    if 'i' in key:
      top_10_movies.append(key)
      if len(top_10_movies) == 10:
        break
  return item_data.loc[top_10_movies]

In [ ]:
return_top_movies(movie_rank)

In [ ]:
user_query = 'u196'
recommendation = nx.pagerank(G, alpha=0.85, personalization={user_query:1})
return_top_movies(recommendation)